In [1]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import numpy as np
from gensim.models.keyedvectors import KeyedVectors
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import math
import requests
from werkzeug.wrappers import Request, Response
from flask import Flask ,redirect ,url_for ,request , render_template, jsonify
from flask_cors import CORS, cross_origin
from gensim.models import KeyedVectors
from functools import lru_cache
from concurrent.futures import ThreadPoolExecutor

In [ ]:
stopwords_path = r'stopwords-en.txt'
with open(stopwords_path, 'r', encoding='utf-8') as fh:
    stopwords = set(fh.read().split(","))


def GrammerChecker(answer):
    req = requests.get("https://api.textgears.com/check.php?text=" + answer + "&key=JmcxHCCPZ7jfXLF6")
    no_of_errors = len(req.json()['errors'])

    #print(no_of_errors)

    if no_of_errors > 5 :
        g = 0
    else:
        g = 1
    return g

def preprocess_text(text):
    # Convert to lowercase and remove punctuation
    text = text.lower()
    text = ''.join(char for char in text if char.isalnum() or char.isspace())
    return text
def tokenize_and_remove_stopwords(text):
    # Tokenize the text and remove stopwords
    tokens = [token for token in text.split() if token not in stopwords]
    return tokens
    
def parallel_keyword_matching(source_doc, target_docs):
    with ThreadPoolExecutor() as executor:
        results = list(executor.map(KeyWordmatching, [source_doc]*len(target_docs), target_docs))
    return results

#key Word matching
def KeyWordmatching(X, Y_lst):
    result = 0
    X_list = tokenize_and_remove_stopwords(preprocess_text(X))
    
    for Y in Y_lst:
        Y_list = tokenize_and_remove_stopwords(preprocess_text(Y))
        intersection = set(X_list).intersection(Y_list)
        cosine = len(intersection) / (len(X_list) * len(Y_list)) ** 0.5
        result += cosine
    
    result /= len(Y_lst)  # Average cosine similarity across target documents
    
    kval = 0
    if cosine > 0.9:
        kval = 1
    elif cosine > 0.8:
        kval = 2
    elif cosine > 0.6:
        kval = 3
    elif cosine > 0.4:
        kval = 4
    elif cosine > 0.2:
        kval = 5
    else:
        kval = 6
    return kval

#length of string
def CheckLength(client_answer):
    
    client_ans = len(client_answer.split())
    #return client_ans
    kval1 = 0
    if client_ans > 50:
        kval1 = 1
    elif client_ans > 40:
        kval1 = 2
    elif client_ans > 30:
        kval1 = 3
    elif client_ans > 20:
        kval1 = 4
    elif client_ans > 10:
        kval1 = 5
    else:
        kval1 = 6
    return kval1

#Synonym

class DocSim:
    def __init__(self, w2v_model, stopwords=None):
        self.w2v_model = w2v_model
        self.stopwords = stopwords if stopwords is not None else []
        
    @lru_cache(maxsize=None)
    def vectorize(self, doc: str) -> np.ndarray:
        """
        Identify the vector values for each word in the given document
        :param doc:
        :return:
        """
        doc = doc.lower()
        words = [w for w in doc.split(" ") if w not in self.stopwords]
        word_vecs = []
        for word in words:
            try:
                vec = self.w2v_model[word]
                word_vecs.append(vec)
            except KeyError:
                # Ignore, if the word doesn't exist in the vocabulary
                pass

        # Assuming that document vector is the mean of all the word vectors
        # PS: There are other & better ways to do it.
        vector = np.mean(word_vecs, axis=0)
        return vector

    def _cosine_sim(self, vecA, vecB):
        """Find the cosine similarity distance between two vectors."""
        csim = np.dot(vecA, vecB) / (np.linalg.norm(vecA) * np.linalg.norm(vecB))
        if np.isnan(np.sum(csim)):
            return 0
        return csim

    def calculate_similarity(self, source_doc, target_docs=None, threshold=0.6):
        """Calculates & returns similarity scores between given source document & all
        the target documents."""
        if not target_docs:
            return []

        if isinstance(target_docs, str):
            target_docs = [target_docs]

        source_vec = self.vectorize(source_doc)
        results = []
        result=[]
        for doc in target_docs:
            target_vec = self.vectorize(doc)
            sim_score = self._cosine_sim(source_vec, target_vec)
            result.append(sim_score)
#             if sim_score > threshold:
#                 results.append({"score": sim_score, "doc": doc})
#             # Sort results by score in desc order
#         results.sort(key=lambda k: k["score"], reverse=True)

        return result



app = Flask(__name__)
CORS(app)
email = "null"
name="null"
roll="null"

@app.route('/')
@cross_origin()
def Base_qstn_paper_set():
    return render_template('index.html')


@app.route('/examinator', methods=['POST', 'GET'])
def examinator():
    if request.method == 'POST':
        data = request.json
        first_list = data.get('first', [])
        second_list = data.get('second',[])
        name = data.get('name')
        roll = data.get('roll')
        email = data.get('emailID')
        print(f"ans_list:{first_list}")
        print(f"ref_ans_list:{second_list}")
        
        googlenews_model_path = r'GoogleNews-vectors-negative300.bin'
        stopwords_path = r'stopwords-en.txt'

        model = KeyedVectors.load_word2vec_format(googlenews_model_path, binary=True)
        with open(stopwords_path, 'r', encoding='utf-8') as fh:
            stopwords = fh.read().split(",")
        ds = DocSim(model, stopwords=stopwords)

        def check(source_docs, target_docs):
            marks = []
            for source_doc, target_doc in zip(source_docs, target_docs):
                if len(source_doc.strip()) == 0:
                    marks.append(0)
                    continue
                sim_scores = ds.calculate_similarity(source_doc, [target_doc])
                similarity_score = sum(sim_scores) / len(sim_scores)
                # Penalize for irrelevant answers
                if similarity_score < 0.6:
                    marks.append(0)
#                     similarity_score=similarity_score-1
                    print(similarity_score)
                    continue
                print(similarity_score)
                key_match = KeyWordmatching(source_doc, [target_doc])
#                 print("key_match:", key_match)
                key_Error = GrammerChecker(source_doc)
#                 print("key_Error:", key_Error)
                key_length = CheckLength(source_doc)
#                 print("key_length:", key_length)
                marks1 = (similarity_score * 6) + (1 / key_match) + (2 * key_Error) + (1 / key_length)
                marks.append(round(marks1))
            return marks

        individual_marks = check(first_list, second_list)
        total_marks = sum(individual_marks)

        response_data = {
            "total_marks": total_marks,
            "individual_marks": individual_marks
        }

        print(f"Total marks: {total_marks}")
        print(f"Individual marks: {individual_marks}")

        return jsonify(response_data)

    return jsonify({"error": "Method not allowed"})




if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('192.168.69.153', 8888, app)


 * Running on http://192.168.69.153:8888
Press CTRL+C to quit


ans_list:['hello']
ref_ans_list:['OS']


192.168.69.153 - - [03/May/2024 13:51:15] "POST /examinator HTTP/1.1" 200 -


0.10797829180955887
Total marks: 0
Individual marks: [0]
ans_list:["Python is a versatile programming language known for its simplicity and flexibility. It is extensively used in web development, data science, machine learning, and automation. Python's rich ecosystem of libraries and frameworks empowers developers to build robust and scalable applications efficiently. Its intuitive syntax and readability contribute to its popularity among developers worldwide.", "Python is a widely-used programming language known for its simplicity and versatility. It is commonly used in web development, data analysis, and machine learning. While Python's syntax is straightforward, it can sometimes be challenging for beginners to grasp certain concepts.", "The sun is a star located at the center of our solar system. It is primarily composed of hydrogen and helium and is the primary source of light and energy for Earth. The sun's energy is generated through nuclear fusion reactions in its core, where hy

192.168.69.153 - - [03/May/2024 13:52:10] "POST /examinator HTTP/1.1" 200 -


0.5442286729812622
Total marks: 17
Individual marks: [10, 7, 0]
ans_list:['']
ref_ans_list:['answer']


192.168.69.153 - - [03/May/2024 13:54:16] "POST /examinator HTTP/1.1" 200 -


Total marks: 0
Individual marks: [0]
